The goal is to write a simple object detection pipeline invoking YOLOX.

In [21]:
import numpy as np
import torch
import torchvision
import os
import cv2
import motmetrics as mm

There are many YOLOX versions based on size, depth and efficiency and we select which yolox version to use by using the `get_exp` function provided by YOLOX.

In [2]:
import sys
sys.path.insert(0, "./YOLOX/")
from YOLOX.yolox.exp.build import get_exp_by_name
from src.detector import Detector

In [3]:
YOLOX_VERSION = "yolox-s" #alternatives: yolox-m, yolox-l, yolox-x, yolox-tiny, yolox-nano, yolov3
yolo_exp = get_exp_by_name(YOLOX_VERSION)
detector = Detector(yolo_exp, filter_classes=None, device=torch.device("cuda"),
                 test_conf=0.25, nms_thres=0.65, class_agnostic=True, 
                 chkpt="yolo_weights/yolox_s.pth", num_classes=80)

In [4]:
img = cv2.imread("img/car_traffic.jpeg")

In [5]:
output, img_info = detector(img)

In [6]:
from src.visualizer import Visualizer

In [7]:
vis = Visualizer()
out_img = vis.add_boxes_to_img(output[0], img_info)

In [8]:
cv2.imwrite("traffic_out.jpg", out_img)

True

In [9]:
cap = cv2.VideoCapture("img/example.mp4")
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) 
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
fps = cap.get(cv2.CAP_PROP_FPS)
while True:
    ret_val, frame = cap.read()
    if ret_val:
        outputs, img_info = detector(frame)
        result_frame = vis.add_boxes_to_img(outputs[0], img_info)
        cv2.namedWindow("yolox", cv2.WINDOW_NORMAL)
        cv2.imshow("yolox", result_frame)
        ch = cv2.waitKey(1)
        if ch == 27 or ch == ord("q") or ch == ord("Q"):
            break
    else:
        break
    
cv2.destroyAllWindows()

In [29]:
test_dir = "data/MOT20/MOT20/train/"
for vid_dir in os.listdir(test_dir):
    print(vid_dir)
    df_rows = []
    for i, file in enumerate(os.listdir(os.path.join(test_dir, vid_dir, "img1"))):
        frame = cv2.imread(os.path.join(test_dir, vid_dir, "img1", file))
        outputs, img_info = detector(frame)
        result_frame = vis.add_boxes_to_img(outputs[0], img_info)
        #cv2.namedWindow("yolox", cv2.WINDOW_NORMAL)
        #cv2.imshow("yolox", result_frame)
        #ch = cv2.waitKey(1)
        #if ch == 27 or ch == ord("q") or ch == ord("Q"):
        #    break
        outputs = outputs[0].cpu()
        outputs /= img_info["ratio"]
        for j, box in enumerate(outputs):
            row = [i, j] + box.numpy()[:4] + [1,-1,-1,-1]
            df_rows.append(row)
            break
#cv2.destroyAllWindows()

MOT20-01


ValueError: operands could not be broadcast together with shapes (2,) (4,) 

In [22]:
for seqname in os.listdir(test_dir):
    

4